In [1]:
import os
import shutil
from pathlib import Path
from ipywidgets import interact

%load_ext autoreload
%autoreload 2
%reload_ext autoreload


from core.Case import *
from core.globals import *
from core.Log import *


from methods.plots import *
from methods.preprocessing import *
from methods.intensity import *
from methods.slicing import *


C:\Users\sulei\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.20) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [49]:
'''
	find out at what point the orientation changes from LAS to RAS
	what vol.data.transpose(2,1,0) does to the shape & orientation
'''
#print(f"  :  {}")

caseID = "TTS_DEM_9902248_72F"
case = Case(caseID)
vol = case.croppedCT
is_label=F
target_spacing=np.array([1.0]*3, dtype=np.float32)
target_shape=(64,64,64)
affine = vol.affine
hu_clip=(-1000.0, 2500.0)
resize_mode="resample",  # {"resample","pad_crop"}
print(f"size/shape: {vol.shape} -  spacing: {vol.spacing}   orientation: {vol.orientation}  ")



size/shape: (328, 284, 212) -  spacing: (0.26171875, 0.26171875, 0.4)   orientation: ('L', 'A', 'S')  


In [ ]:
interp = sitk.sitkNearestNeighbor if is_label else sitk.sitkLinear
img = sitk.GetImageFromArray(vol.data.transpose(2,1,0))
img.SetDirection(tuple(vol.affine[:3, :3].flatten()))   				# Set orientation
img.SetOrigin(tuple(vol.affine[:3, 3]))                  				# Set origin
img.SetSpacing(tuple(np.sqrt((vol.affine[:3, :3]**2).sum(axis=0))))

# --- 1) Clip HU (for CT only)
#if not is_label:
img = sitk.Cast(img, sitk.sitkFloat32) # ensure floating point to avoid overflow surprises on clamp
img = sitk.Clamp(img, lowerBound=float(hu_clip[0]), upperBound=float(hu_clip[1]))

# --- 2) Resample to fixed spacing (physics-aware)
img_size = np.array(img.GetSize())
img_spacing = np.array(img.GetSpacing())
#resample.SetSize([int(s) for s in new_size])
new_size1 = (img_size * (img_spacing / np.array(target_spacing))).astype(int)

# derive new voxel counts to keep physical extent constant

# (167, 136, 41) x (0.68, 0.68, 2.0)
phys_size = img_size * img_spacing          # mm per axis
new_size = np.maximum(np.round(phys_size / target_spacing), 1).astype(int)
#print(f"newsize: {new_size}, newsize1: {new_size1}")  # seems minimal difference


resample = sitk.ResampleImageFilter()
resample.SetSize(new_size.tolist())
resample.SetOutputSpacing(target_spacing.tolist())
resample.SetOutputDirection(img.GetDirection())
resample.SetOutputOrigin(img.GetOrigin())
resample.SetInterpolator(interp)
img = resample.Execute(img)
print(f"Size: {img.GetSize()}, Spacing: {img.GetSpacing()}")


newsize: [86 74 85], newsize1: [85 74 84]
Size: (86, 74, 85), Spacing: (1.0, 1.0, 1.0)


In [51]:
# --- 3) Fix final shape to target_shape=64³
#if tuple(img.GetSize()) != tuple(target_shape):
#if resize_mode == "resample": for intensity values, not labels/masks
# resample again to exact voxel counts, preserving physical field of view FOV
cur_size = np.array(img.GetSize(), dtype=np.int64)
cur_spacing = np.array(img.GetSpacing(), dtype=np.float32)
cur_phys = cur_size * cur_spacing
out_size = np.array(target_shape, dtype=int)
out_spacing = (cur_phys / out_size).astype(np.float32)
rs2 = sitk.ResampleImageFilter()
rs2.SetSize(out_size.tolist())
rs2.SetOutputSpacing(out_spacing.tolist())
rs2.SetOutputDirection(img.GetDirection())
rs2.SetOutputOrigin(img.GetOrigin())
rs2.SetInterpolator(interp)
img = rs2.Execute(img)
print(f"Size: {img.GetSize()}, Spacing: {img.GetSpacing()}")



Size: (64, 64, 64), Spacing: (1.34375, 1.15625, 1.328125)


In [ ]:
#if resize_mode == "pad_crop":
# symmetric pad or center-crop without further interpolation
# (spacing unchanged; FOV changes)
interp = sitk.sitkNearestNeighbor
from math import floor
def pad_to(img, target_shape):
	size = np.array(img.GetSize())
	pad_lower = np.maximum((np.array(target_shape) - size)//2, 0)
	pad_upper = np.maximum(np.array(target_shape) - size - pad_lower, 0)
	return sitk.ConstantPad(img,
	                        padList=pad_lower.tolist(),
	                        padUpperBound=pad_upper.tolist(),
	                        constant=0 if not is_label else 0)
def crop_to(img, target_shape):
	size = np.array(img.GetSize())
	start = np.maximum((size - np.array(target_shape))//2, 0).astype(int)
	extractor = sitk.RegionOfInterestImageFilter()
	extractor.SetIndex(start.tolist())
	extractor.SetSize(np.minimum(size, np.array(target_shape)).astype(int).tolist())
	out = extractor.Execute(img)
	if tuple(out.GetSize()) != tuple(target_shape):
		out = pad_to(out, target_shape)  # pad if we cropped too tight on an axis
	return out
# decide per axis
size_now = np.array(img.GetSize())
if np.any(size_now < np.array(target_shape)):
	img = pad_to(img, target_shape)
if tuple(img.GetSize()) != tuple(target_shape):
	img = crop_to(img, target_shape)
#else:
#raise ValueError("resize_mode must be 'resample' or 'pad_crop'.")


In [ ]:
caseID="TTS_BLT_11022191_60F"
case = Case(caseID)
segs = case.resampledsegs
path = f"{case.casePath}/onehot_mask.nii.gz"
onhot = case.onehot_mask(segs, path)
print(f"onhot shape: {onhot.shape}, orientation: {onhot.orientation}")


In [ ]:
'''
[WARNING] Missing fullCT.nii.gz in CNTRL_KMC_92706_81F
[WARNING] Missing fullCT.nii.gz in TTS_CDA_38093134_74F
[WARNING] Missing fullCT.nii.gz in TTS_GA_39800503_22F
[WARNING] Missing fullCT.nii.gz in TTS_LW_12109724_62F
[WARNING] Missing fullCT.nii.gz in TTS_MA_6894703_78F
[WARNING] Missing fullCT.nii.gz in TTS_SSA_35792472_73F
'''
